In [ ]:
!pip install contractions
!pip install unidecode

     |████████████████████████████████| 327kB 3.9MB/s 
     |████████████████████████████████| 286kB 20.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85404 sha256=480541538dc7d3e92e0995f6d762267dfa8d1cc06a3533d86f83842a4c13351f
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
     |████████████████████████████████| 245kB 4.0MB/s 


In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torchtext
import random
import time
import torch.nn.functional as F
import re
import spacy
from unidecode import unidecode
import contractions
from torchtext.legacy.data import TabularDataset,BucketIterator,get_tokenizer,Field,LabelField

In [ ]:
dirs="/content/drive/MyDrive/dataset/Bcc_news_classification/bbc-fulltext (document classification)/bbc"

In [ ]:
Dir="/content/drive/MyDrive/dataset/Bcc_news_classification/bbc-fulltext (document classification)/bbc"

In [ ]:
texts=[]
labels=[]
for folder in os.listdir(Dir):
  if "TXT" in folder:
    continue
  paths=os.path.join(Dir,folder)
  for path in os.listdir(paths):
    # print(paths+"/"+path)
    with open(paths+"/"+path,"r", encoding = "latin") as file:
      data=file.read()
      data = data.replace("\n"," ").replace('\r','')
      texts.append(data)
      file.close() # file is being closed
#     letting the category of the news as the folder name in which it reside
      labels.append(folder)

In [ ]:
df=pd.DataFrame({"text":texts,"label":labels})

In [ ]:
df.head()

,text,label
0,Yukos unit buyer faces loan claim The owners ...,business
1,High fuel prices hit BA's profits British Air...,business
2,Dollar gains on Greenspan speech The dollar h...,business
3,Ad sales boost Time Warner profit Quarterly p...,business
4,Japan narrowly escapes recession Japan's econ...,business


In [ ]:
df["label"].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: label, dtype: int64

In [ ]:
df["text"]=df["text"].apply(lambda x:x.lower())

In [ ]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
def expend_contraction(txt):
  list_contraction=[]
  for word in txt.split():
    list_contraction.append(contractions.fix(word))

  expend_word=" ".join(list_contraction)
  return expend_word

In [ ]:
df["text"]=df["text"].apply(lambda x:expend_contraction(x))

In [ ]:
df["text"]=df["text"].apply(lambda x:unidecode(x))

In [ ]:
df["text"]=df["text"].apply(lambda x:re.sub(r'http\S+', '', x))

In [ ]:
df["text"]=df["text"].apply(lambda x:re.sub('[^A-Za-z0-9]+', ' ', x))

In [ ]:
df.head()

,text,label
0,yukos unit buyer faces loan claim the owners o...,business
1,high fuel prices hit ba s profits british airw...,business
2,dollar gains on greenspan speech the dollar ha...,business
3,ad sales boost time warner profit quarterly pr...,business
4,japan narrowly escapes recession japan s econo...,business


In [ ]:
df.to_csv("/content/drive/MyDrive/dataset/Bcc_news_classification/bbc-fulltext (document classification)/All_data.csv",index=False)

In [ ]:
TEXT=Field(tokenize="spacy",tokenizer_language='en_core_web_sm')
LABEL=LabelField()

In [ ]:
field=[("text",TEXT),("label",LABEL)]

In [ ]:
training_data=TabularDataset(path="/content/drive/MyDrive/dataset/Bcc_news_classification/bbc-fulltext (document classification)/All_data.csv",
                             format="csv",fields=field,skip_header=True)

print(vars(training_data[3]))

{'text': ['ad', 'sales', 'boost', 'time', 'warner', 'profit', 'quarterly', 'profits', 'at', 'us', 'media', 'giant', 'timewarner', 'jumped', '76', 'to', '1', '13bn', 'aPS600', 'm', 'for', 'the', 'three', 'months', 'to', 'december', 'from', '639', 'm', 'year', 'earlier', 'the', 'firm', 'which', 'is', 'now', 'one', 'of', 'the', 'biggest', 'investors', 'in', 'google', 'benefited', 'from', 'sales', 'of', 'high', 'speed', 'internet', 'connections', 'and', 'higher', 'advert', 'sales', 'timewarner', 'said', 'fourth', 'quarter', 'sales', 'rose', '2', 'to', '11', '1bn', 'from', '10', '9bn', 'its', 'profits', 'were', 'buoyed', 'by', 'one', 'off', 'gains', 'which', 'offset', 'a', 'profit', 'dip', 'at', 'warner', 'bros', 'and', 'less', 'users', 'for', 'aol', 'time', 'warner', 'said', 'on', 'friday', 'that', 'it', 'now', 'owns', '8', 'of', 'search', 'engine', 'google', 'but', 'its', 'own', 'internet', 'business', 'aol', 'had', 'has', 'mixed', 'fortunes', 'it', 'lost', '464', '000', 'subscribers', 'i

In [ ]:
len(training_data)

In [ ]:
train_data,valid_data=training_data.split(random_state=random.SEED(2020))

In [ ]:
len(train_data),len(valid_data)

(1558, 667)

In [ ]:
TEXT.build_vocab(train_data,max_size=25_000,vectors="glove.6B.100d",unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.37MB/s]                          
100%|█████████▉| 399943/400000 [00:18<00:00, 20350.40it/s]

In [ ]:
LABEL.vocab.stoi

defaultdict(None,
            {'business': 0,
             'entertainment': 3,
             'politics': 2,
             'sport': 1,
             'tech': 4})

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_iterator,valid_iterator=BucketIterator.splits((train_data,valid_data),
                                                    sort_key=lambda x:len(x.text),
                                                    batch_size=64,
                                                    device=device)

In [ ]:
class CNN(nn.Module):
  def __init__(self,vocab_size,embedding_dim,output_dim,n_filers,filter_size,dropout,pad_token):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,embedding_dim,padding_idx=pad_token)
    self.convs=nn.ModuleList([nn.Conv2d(in_channels=1,out_channels=n_filers,kernel_size=(fs,embedding_dim))
                              for fs in filter_size])
    
    self.fc=nn.Linear(len(filter_size)*n_filter,output_dim)
    self.dropout=nn.Dropout(dropout)

  def forward(self,text):
    embedded=self.embedding(text)
    embedded=embedded.permute(1,0,2)
    embedded=embedded.unsqueeze(1)

    #Applying convs layers
    conved=[F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
    #Applying pooling layer
    pooled=[F.max_pool1d(conv,conv.shape[2]).squeeze(2) for conv in conved]
    #pooled_n=[batch_size,n_filters]
    cat=self.dropout(torch.cat(pooled,dim=1))
    #cat = [batch_size,n_filers*len(filter_size)]
    return self.fc(cat)

In [ ]:
vocab_size=len(TEXT.vocab)
embedding_dim=100
output_dim=len(LABEL.vocab)
n_filter=100
filter_size=[2,3,4]
dropout=0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
model=CNN(vocab_size,embedding_dim,output_dim,n_filter,filter_size,dropout,PAD_IDX)

In [ ]:
model

CNN(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
  )
  (fc): Linear(in_features=300, out_features=5, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
for i in train_iterator:
  txt=i.text
  print(model(txt))
  break

tensor([[ 0.8455, -0.3125,  0.5106,  0.4141, -2.2318],
        [-0.2418, -0.9268,  0.1501, -0.5692, -2.0717],
        [-0.7502, -0.3452, -0.0845,  2.3658, -2.0979],
        [ 0.5622,  0.4539, -0.8483,  1.1220, -2.0278],
        [ 0.6459, -0.8759, -0.9832,  1.2236,  0.1686],
        [-0.8726, -0.6120,  0.1146, -0.4009, -1.6033],
        [-0.0183,  0.8172,  1.4335, -0.4170, -0.3065],
        [-0.3086,  0.9753, -0.2562, -0.5305, -2.3628],
        [-0.2682, -0.1718,  0.1450,  0.5972, -1.5889],
        [-0.7427, -0.5444,  1.1546, -0.5443, -2.3384],
        [-0.2791, -0.0228,  0.2173,  2.3554, -2.8607],
        [-1.5650,  0.9309,  0.0828,  1.1966, -1.5249],
        [-0.2376, -1.1564,  1.6057,  1.3830, -2.0793],
        [-2.3915, -0.6164,  0.6787, -1.9380, -2.6051],
        [-0.6366, -0.9833,  0.8152,  1.0139, -2.2346],
        [-2.0453, -0.6666,  2.2154, -0.5470, -0.6414],
        [ 0.4577, -0.0276,  1.0864,  1.1170, -2.2631],
        [-3.0178, -0.2692,  0.3095,  0.8076, -0.4902],
        [ 

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,005 trainable parameters


In [ ]:

pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.4176,  0.6554,  1.4659,  ..., -1.2013, -2.4477, -0.4971],
        [-1.4233, -0.9832,  0.7436,  ..., -0.2335, -0.0075,  1.8779],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.4314, -0.1521,  0.2078,  ..., -0.0066, -0.0789,  0.6983],
        [-0.1778, -0.0117,  0.3881,  ..., -0.1235, -0.2461, -0.2537],
        [ 0.3933,  0.4678, -0.4488,  ...,  0.3512,  0.0777,  0.2561]])

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embedding.weight.data[PAD_IDX] = torch.zeros(embedding_dim)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    # start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # end_time = time.time()

    # epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), dirs+'Multi-class_Sentiment_Analysis.pt')
    
    # print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

	Train Loss: 1.392 | Train Acc: 43.80%
	 Val. Loss: 0.922 |  Val. Acc: 88.99%
	Train Loss: 0.747 | Train Acc: 82.34%
	 Val. Loss: 0.432 |  Val. Acc: 96.73%
	Train Loss: 0.362 | Train Acc: 93.07%
	 Val. Loss: 0.224 |  Val. Acc: 97.02%
	Train Loss: 0.209 | Train Acc: 95.81%
	 Val. Loss: 0.149 |  Val. Acc: 97.44%
	Train Loss: 0.146 | Train Acc: 96.13%
	 Val. Loss: 0.120 |  Val. Acc: 97.73%


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [ ]:
pred_class = predict_class(model, "i love this film")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 3 = entertainment


In [ ]:
pred_class = predict_class(model, "i love playing cricket")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 1 = sport


In [ ]:
pred_class = predict_class(model, "google search")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 4 = tech


In [ ]:
pred_class = predict_class(model, "Sports is the ultimate destination for Sports fans from around the World")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 1 = sport


In [ ]:
pred_class = predict_class(model, "market today, stock, economy news from India")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 0 = business


In [ ]:
red_class = predict_class(model, " stay connected with BloombergQuint. Be up to date with today's business news headlines, finance bulletin")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 0 = business


In [ ]:
pred_class = predict_class(model, "The protests were held in Dum Dum, Central Avenue and Chetla areas of Kolkata, Canning in South 24 Parganas, Chinsurah in Hooghly, and Malda, besides")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 2 = politics


In [ ]:
pred_class = predict_class(model, ("The Family Man 2's Dhriti actor Ashlesha Thakur gets 'rishtas' in her DMs".lower()))
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 3 = entertainment


In [ ]:
pred_class = predict_class(model, ("PUBG Mobile World Invitational 2021 announced, there’s $3 million charity prize pool involved").lower())
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 1 = sport


In [ ]:
LABEL.vocab.stoi

defaultdict(None,
            {'business': 0,
             'entertainment': 3,
             'politics': 2,
             'sport': 1,
             'tech': 4})